In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, TimeDistributed, Bidirectional, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Embedding
from sklearn.metrics import classification_report
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from itertools import product
import pandas as pd
from keras.models import Model
import numpy as np
import seqdata

tf.random.set_seed(7)

2023-04-03 07:26:49.555780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Plot model
#
# tf.keras.utils.plot_model(
#     model,
#     to_file='model.png',
#     show_shapes=False,
#     show_dtype=False,
#     show_layer_names=True,
#     rankdir='TB',
#     expand_nested=False,
#     dpi=96,
#     layer_range=None,
#     show_layer_activations=False
# )

## One-hot encoding

In [18]:
train_data = seqdata.Seq('train/', 'ohe')
test_data = seqdata.Seq('test/', 'ohe')

max_len = seqdata.pad_data(train_data, test_data)

train_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], True)
test_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], False)

### CNN

In [34]:
# Input layer
input_layer = Input(shape=(max_len, 4))

# CNN layers
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv1)
dropout1 = Dropout(0.5)(conv2)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
flatten1 = Flatten()(max_pool1)

# Dense layers
dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(flatten1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 3339, 4)]         0         
                                                                 
 conv1d_21 (Conv1D)          (None, 3337, 128)         1664      
                                                                 
 conv1d_22 (Conv1D)          (None, 3335, 128)         49280     
                                                                 
 dropout_25 (Dropout)        (None, 3335, 128)         0         
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 1667, 128)        0         
 g1D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 213376)            0         
                                                          

In [35]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 96s 1s/step - loss: 1.6820 - Precision: 0.6939 - val_loss: 2.5995 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 90s 1s/step - loss: 1.0740 - Precision: 0.8282 - val_loss: 2.7728 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 87s 1s/step - loss: 0.8507 - Precision: 0.8762 - val_loss: 1.5330 - val_Precision: 0.7316 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 92s 1s/step - loss: 0.7418 - Precision: 0.8972 - val_loss: 2.1328 - val_Precision: 0.5093 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - ETA: 0s - loss: 0.6831 - Precision: 0.9019
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 93s 1s/step - loss: 0.6831 - Precision: 0.9019 - val_loss: 1.7362 - val_Precision: 0.7123 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - 88s 

In [36]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 4s 201ms/step


,precision,recall,f1-score,support
snoRNA,0.617647,0.600000,0.608696,35.000000
rRNA,0.895833,0.934783,0.914894,138.000000
snRNA,0.800000,0.363636,0.500000,22.000000
mRNA,0.870000,0.935484,0.901554,186.000000
pre_miRNA,0.600000,0.660000,0.628571,50.000000
miRNA,0.456140,0.500000,0.477064,52.000000
tRNA,0.977273,0.966292,0.971751,89.000000
tmRNA,1.000000,0.771429,0.870968,70.000000
accuracy,0.827103,0.827103,0.827103,0.827103
macro avg,0.777112,0.716453,0.734187,642.000000


### CNN + LSTM

In [6]:
# Input layer
input_layer = Input(shape=(max_len, 4))

# CNN layers
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)

lstm1 = LSTM(128)(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(lstm1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3339, 4)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 3337, 128)         1664      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1668, 128)        0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                           

In [7]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 178s 3s/step - loss: 2.0002 - Precision: 0.0000e+00 - val_loss: 2.9314 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 165s 3s/step - loss: 1.8873 - Precision: 0.0000e+00 - val_loss: 3.5331 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - ETA: 0s - loss: 1.8887 - Precision: 0.0000e+00
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 163s 2s/step - loss: 1.8887 - Precision: 0.0000e+00 - val_loss: 3.5118 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 167s 3s/step - loss: 1.8746 - Precision: 0.0000e+00 - val_loss: 3.6070 - val_Precision: 0.0000e+00 - lr: 2.0000e-04
Epoch 5/10
66/66 [==============================] - ETA: 0s - loss: 1.8768 - Precision: 0.0000e+00
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.

In [8]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 13s 581ms/step


/home/brenoslivio/miniconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brenoslivio/miniconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brenoslivio/miniconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

,precision,recall,f1-score,support
snoRNA,0.000000,0.000000,0.000000,35.000000
rRNA,0.214953,1.000000,0.353846,138.000000
snRNA,0.000000,0.000000,0.000000,22.000000
mRNA,0.000000,0.000000,0.000000,186.000000
pre_miRNA,0.000000,0.000000,0.000000,50.000000
miRNA,0.000000,0.000000,0.000000,52.000000
tRNA,0.000000,0.000000,0.000000,89.000000
tmRNA,0.000000,0.000000,0.000000,70.000000
accuracy,0.214953,0.214953,0.214953,0.214953
macro avg,0.026869,0.125000,0.044231,642.000000


### CNN + BiLSTM

In [3]:
# Input layer
input_layer = Input(shape=(max_len, 4))

# CNN layers
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)

lstm1 = Bidirectional(LSTM(128))(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(lstm1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

2023-04-02 08:21:04.705592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 08:21:04.707393: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3339, 4)]         0         
                                                                 
 conv1d (Conv1D)             (None, 3337, 128)         1664      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1668, 128)        0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0     

In [4]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 351s 5s/step - loss: 1.9106 - Precision: 1.0000 - val_loss: 3.1448 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 333s 5s/step - loss: 1.6422 - Precision: 0.5028 - val_loss: 3.2840 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - ETA: 0s - loss: 1.4082 - Precision: 0.6483
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 362s 5s/step - loss: 1.4082 - Precision: 0.6483 - val_loss: 4.5461 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 410s 6s/step - loss: 1.3467 - Precision: 0.6285 - val_loss: 2.7725 - val_Precision: 0.0000e+00 - lr: 2.0000e-04
Epoch 5/10
66/66 [==============================] - 354s 5s/step - loss: 1.1165 - Precision: 0.7237 - val_loss: 3.0243 - val_Precision: 0.0000e+00 - lr: 2.0000e-04
Epoch 6/10
66/66 [=============

In [5]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 23s 1s/step


,precision,recall,f1-score,support
snoRNA,0.357143,0.428571,0.389610,35.000000
rRNA,0.621891,0.905797,0.737463,138.000000
snRNA,0.250000,0.045455,0.076923,22.000000
mRNA,0.829016,0.860215,0.844327,186.000000
pre_miRNA,0.532258,0.660000,0.589286,50.000000
miRNA,0.413793,0.230769,0.296296,52.000000
tRNA,0.768519,0.932584,0.842640,89.000000
tmRNA,1.000000,0.042857,0.082192,70.000000
accuracy,0.672897,0.672897,0.672897,0.672897
macro avg,0.596577,0.513281,0.482342,642.000000


### CNN + Features

In [52]:
input_layer = Input(shape=(max_len, 4))

conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv1)
dropout1 = Dropout(0.5)(conv2)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
cnn_out = Flatten()(max_pool1)

# feature extraction input

feat_extraction_input = Input(shape=(426,))
feat_extraction_out = Flatten()(feat_extraction_input)

concat = Concatenate()([cnn_out, feat_extraction_out])

dense1 = Dense(128, activation='relu')(concat)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=[input_layer, feat_extraction_input], outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 3339, 4)]    0           []                               
                                                                                                  
 conv1d_38 (Conv1D)             (None, 3337, 128)    1664        ['input_28[0][0]']               
                                                                                                  
 conv1d_39 (Conv1D)             (None, 3335, 128)    49280       ['conv1d_38[0][0]']              
                                                                                                  
 dropout_42 (Dropout)           (None, 3335, 128)    0           ['conv1d_39[0][0]']              
                                                                                           

In [53]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit([train_data.seqs, train_data.features], train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 78s 1s/step - loss: 0.8941 - Precision: 0.8216 - val_loss: 1.3483 - val_Precision: 0.5823 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 81s 1s/step - loss: 0.4428 - Precision: 0.8998 - val_loss: 1.0185 - val_Precision: 0.7968 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 82s 1s/step - loss: 0.3192 - Precision: 0.9107 - val_loss: 0.7495 - val_Precision: 0.8818 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 78s 1s/step - loss: 0.2352 - Precision: 0.9354 - val_loss: 0.7101 - val_Precision: 0.8455 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 80s 1s/step - loss: 0.1779 - Precision: 0.9441 - val_loss: 1.1154 - val_Precision: 0.7665 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - ETA: 0s - loss: 0.1474 - Precision: 0.9530
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 74s 1s/step 

In [55]:
# Model prediction
model_pred = model.predict([test_data.seqs, test_data.features])
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 4s 199ms/step


,precision,recall,f1-score,support
snoRNA,0.575000,0.657143,0.613333,35.000000
rRNA,0.894737,0.985507,0.937931,138.000000
snRNA,0.666667,0.454545,0.540541,22.000000
mRNA,0.983333,0.951613,0.967213,186.000000
pre_miRNA,0.666667,0.720000,0.692308,50.000000
miRNA,0.549020,0.538462,0.543689,52.000000
tRNA,1.000000,0.988764,0.994350,89.000000
tmRNA,1.000000,0.885714,0.939394,70.000000
accuracy,0.872274,0.872274,0.872274,0.872274
macro avg,0.791928,0.772719,0.778595,642.000000


### CNN + BiLSTM + Features

In [19]:
input_layer = Input(shape=(max_len, 4))

conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
lstm1 = Bidirectional(LSTM(128))(max_pool1)
cnn_out = Flatten()(lstm1)

# feature extraction input

feat_extraction_input = Input(shape=(426,))
feat_extraction_out = Flatten()(feat_extraction_input)

concat = Concatenate()([cnn_out, feat_extraction_out])

dense1 = Dense(128, activation='relu')(concat)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=[input_layer, feat_extraction_input], outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 3339, 4)]    0           []                               
                                                                                                  
 conv1d_8 (Conv1D)              (None, 3337, 128)    1664        ['input_11[0][0]']               
                                                                                                  
 dropout_6 (Dropout)            (None, 3337, 128)    0           ['conv1d_8[0][0]']               
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 1668, 128)   0           ['dropout_6[0][0]']              
                                                                                            

In [20]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit([train_data.seqs, train_data.features], train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 384s 6s/step - loss: 1.0778 - Precision: 0.7906 - val_loss: 1.6267 - val_Precision: 0.3000 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 402s 6s/step - loss: 0.6550 - Precision: 0.8628 - val_loss: 1.2707 - val_Precision: 0.6786 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 402s 6s/step - loss: 0.5166 - Precision: 0.8775 - val_loss: 0.9230 - val_Precision: 0.8012 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 402s 6s/step - loss: 0.4291 - Precision: 0.9067 - val_loss: 0.7678 - val_Precision: 0.8450 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 402s 6s/step - loss: 0.3880 - Precision: 0.9082 - val_loss: 0.7421 - val_Precision: 0.8439 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - 368s 6s/step - loss: 0.3319 - Precision: 0.9238 - val_loss: 0.7607 - val_Precision: 0.8454 - lr: 0.0010
Epoch 7/10
66/66 [==============================] - 342s 5s/step

In [21]:
# Model prediction
model_pred = model.predict([test_data.seqs, test_data.features])
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 27s 1s/step


,precision,recall,f1-score,support
snoRNA,0.527778,0.542857,0.535211,35.000000
rRNA,0.964539,0.985507,0.974910,138.000000
snRNA,0.478261,0.500000,0.488889,22.000000
mRNA,0.989011,0.967742,0.978261,186.000000
pre_miRNA,0.622951,0.760000,0.684685,50.000000
miRNA,0.531915,0.480769,0.505051,52.000000
tRNA,0.966667,0.977528,0.972067,89.000000
tmRNA,1.000000,0.885714,0.939394,70.000000
accuracy,0.869159,0.869159,0.869159,0.869159
macro avg,0.760140,0.762515,0.759808,642.000000


## Label Encoding (Embedding Layer)

In [9]:
train_data = seqdata.Seq('train/', 'label')
test_data = seqdata.Seq('test/', 'label')

max_len = seqdata.pad_data(train_data, test_data)

train_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], True)
test_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], False)

### CNN

In [3]:
# Functional Model Keras

# cnn input

input_layer = Input(shape=(max_len,))

embedding1 = Embedding(5, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding1)
conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv1)
dropout1 = Dropout(0.5)(conv2)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
flatten1 = Flatten()(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(flatten1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3339)]            0         
                                                                 
 embedding (Embedding)       (None, 3339, 32)          160       
                                                                 
 conv1d (Conv1D)             (None, 3337, 128)         12416     
                                                                 
 conv1d_1 (Conv1D)           (None, 3335, 128)         49280     
                                                                 
 dropout (Dropout)           (None, 3335, 128)         0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1667, 128)        0         
 )                                                               
                                                             

2023-04-01 21:17:24.654617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 21:17:24.656675: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 79s 1s/step - loss: 1.8055 - Precision: 0.6178 - val_loss: 2.1459 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 76s 1s/step - loss: 1.0946 - Precision: 0.7744 - val_loss: 2.4530 - val_Precision: 0.1829 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 76s 1s/step - loss: 0.9036 - Precision: 0.8396 - val_loss: 1.3871 - val_Precision: 0.8038 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 76s 1s/step - loss: 0.8407 - Precision: 0.8670 - val_loss: 1.9194 - val_Precision: 0.7073 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 76s 1s/step - loss: 0.7649 - Precision: 0.8830 - val_loss: 1.3552 - val_Precision: 0.8433 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - 76s 1s/step - loss: 0.7050 - Precision: 0.9132 - val_loss: 1.7540 - val_Precision: 0.7489 - lr: 0.0010
Epoch 7/10
66/66 [==============================] - ETA: 0s - loss

In [5]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 4s 190ms/step


,precision,recall,f1-score,support
snoRNA,0.527778,0.542857,0.535211,35.000000
rRNA,0.882759,0.927536,0.904594,138.000000
snRNA,0.692308,0.409091,0.514286,22.000000
mRNA,0.901554,0.935484,0.918206,186.000000
pre_miRNA,0.661017,0.780000,0.715596,50.000000
miRNA,0.509804,0.500000,0.504854,52.000000
tRNA,1.000000,0.966292,0.982857,89.000000
tmRNA,1.000000,0.842857,0.914729,70.000000
accuracy,0.841121,0.841121,0.841121,0.841121
macro avg,0.771902,0.738015,0.748792,642.000000


### CNN + BiLSTM

In [20]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(5, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding1)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)

lstm1 = Bidirectional(LSTM(128))(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(lstm1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 3339)]            0         
                                                                 
 embedding_7 (Embedding)     (None, 3339, 128)         640       
                                                                 
 conv1d_6 (Conv1D)           (None, 3337, 128)         49280     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 1668, 128)        0         
 1D)                                                             
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 128)               3289

In [21]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 362s 5s/step - loss: 1.8975 - Precision: 0.1818 - val_loss: 3.0821 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 370s 6s/step - loss: 1.6300 - Precision: 0.5710 - val_loss: 3.1638 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 373s 6s/step - loss: 1.4222 - Precision: 0.6360 - val_loss: 2.5038 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 370s 6s/step - loss: 1.2432 - Precision: 0.7131 - val_loss: 3.0273 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - ETA: 0s - loss: 1.2333 - Precision: 0.6826
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 377s 6s/step - loss: 1.2333 - Precision: 0.6826 - val_loss: 4.0192 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 6/10
66/66 [=====================

In [22]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 24s 1s/step


,precision,recall,f1-score,support
snoRNA,0.333333,0.400000,0.363636,35.000000
rRNA,0.563063,0.905797,0.694444,138.000000
snRNA,0.500000,0.045455,0.083333,22.000000
mRNA,0.819277,0.731183,0.772727,186.000000
pre_miRNA,0.555556,0.800000,0.655738,50.000000
miRNA,0.333333,0.192308,0.243902,52.000000
tRNA,0.865979,0.943820,0.903226,89.000000
tmRNA,1.000000,0.157143,0.271605,70.000000
accuracy,0.655763,0.655763,0.655763,0.655763
macro avg,0.621318,0.521963,0.498577,642.000000


### CNN + Features

In [13]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(5, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(embedding1)
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv1)
dropout1 = Dropout(0.5)(conv2)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
cnn_out = Flatten()(max_pool1)

# feature extraction input

feat_extraction_input = Input(shape=(426,))
feat_extraction_out = Flatten()(feat_extraction_input)

concat = Concatenate()([cnn_out, feat_extraction_out])

dense = Dense(128, activation='relu')(concat)

output_layer = Dense(8, activation='softmax')(dense)

model = Model(inputs=[input_layer, feat_extraction_input], outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 3339)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 3339, 32)     160         ['input_5[0][0]']                
                                                                                                  
 conv1d_4 (Conv1D)              (None, 3337, 64)     6208        ['embedding_1[0][0]']            
                                                                                                  
 dropout_3 (Dropout)            (None, 3337, 64)     0           ['conv1d_4[0][0]']               
                                                                                            

In [11]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit([train_data.seqs, train_data.features], train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 33s 482ms/step - loss: 0.7047 - Precision: 0.8752 - val_loss: 1.1062 - val_Precision: 0.7622 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 28s 430ms/step - loss: 0.3585 - Precision: 0.9089 - val_loss: 0.7740 - val_Precision: 0.8706 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 37s 559ms/step - loss: 0.2497 - Precision: 0.9216 - val_loss: 0.6593 - val_Precision: 0.8698 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 36s 542ms/step - loss: 0.1785 - Precision: 0.9463 - val_loss: 1.0476 - val_Precision: 0.7655 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - ETA: 0s - loss: 0.1330 - Precision: 0.9577
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 35s 535ms/step - loss: 0.1330 - Precision: 0.9577 - val_loss: 0.9569 - val_Precision: 0.8190 - lr: 0.0010
Epoch 6/10
66/66 [==============================]

In [12]:
# Model prediction
model_pred = model.predict([test_data.seqs, test_data.features])
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 3s 115ms/step


,precision,recall,f1-score,support
snoRNA,0.641026,0.714286,0.675676,35.000000
rRNA,0.925170,0.985507,0.954386,138.000000
snRNA,0.529412,0.409091,0.461538,22.000000
mRNA,0.967391,0.956989,0.962162,186.000000
pre_miRNA,0.645833,0.620000,0.632653,50.000000
miRNA,0.517241,0.576923,0.545455,52.000000
tRNA,1.000000,0.988764,0.994350,89.000000
tmRNA,1.000000,0.871429,0.931298,70.000000
accuracy,0.869159,0.869159,0.869159,0.869159
macro avg,0.778259,0.765374,0.769690,642.000000


### CNN + BiLSTM + Features

In [15]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(5, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(embedding1)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
lstm1 = Bidirectional(LSTM(128))(max_pool1)
cnn_out = Flatten()(lstm1)

# feature extraction input

feat_extraction_input = Input(shape=(426,))
feat_extraction_out = Flatten()(feat_extraction_input)

concat = Concatenate()([cnn_out, feat_extraction_out])

dense = Dense(128, activation='relu')(concat)

output_layer = Dense(8, activation='softmax')(dense)

model = Model(inputs=[input_layer, feat_extraction_input], outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 3339)]       0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 3339, 32)     160         ['input_9[0][0]']                
                                                                                                  
 conv1d_7 (Conv1D)              (None, 3337, 64)     6208        ['embedding_3[0][0]']            
                                                                                                  
 dropout_5 (Dropout)            (None, 3337, 64)     0           ['conv1d_7[0][0]']               
                                                                                            

In [16]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit([train_data.seqs, train_data.features], train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 321s 5s/step - loss: 0.7596 - Precision: 0.8702 - val_loss: 1.1300 - val_Precision: 0.7639 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 342s 5s/step - loss: 0.4016 - Precision: 0.9020 - val_loss: 0.8535 - val_Precision: 0.8878 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 330s 5s/step - loss: 0.3181 - Precision: 0.9252 - val_loss: 0.6643 - val_Precision: 0.9150 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 300s 5s/step - loss: 0.2685 - Precision: 0.9284 - val_loss: 0.6185 - val_Precision: 0.9082 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 381s 6s/step - loss: 0.2252 - Precision: 0.9399 - val_loss: 0.6522 - val_Precision: 0.8990 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - ETA: 0s - loss: 0.1829 - Precision: 0.9562
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 370s 6s

In [17]:
# Model prediction
model_pred = model.predict([test_data.seqs, test_data.features])
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 21s 943ms/step


,precision,recall,f1-score,support
snoRNA,0.542857,0.542857,0.542857,35.000000
rRNA,0.950704,0.978261,0.964286,138.000000
snRNA,0.523810,0.500000,0.511628,22.000000
mRNA,0.988889,0.956989,0.972678,186.000000
pre_miRNA,0.685185,0.740000,0.711538,50.000000
miRNA,0.509091,0.538462,0.523364,52.000000
tRNA,0.956522,0.988764,0.972376,89.000000
tmRNA,1.000000,0.900000,0.947368,70.000000
accuracy,0.870717,0.870717,0.870717,0.870717
macro avg,0.769632,0.768167,0.768262,642.000000


## k-mer Encoding (Embedding layer)

In [2]:
train_data = seqdata.Seq('train/', 'k-mer')
test_data = seqdata.Seq('test/', 'k-mer')

max_len = seqdata.pad_data(train_data, test_data)

train_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], True)
test_data.feature_extraction([1, 2, 3, 4, 5, 6, 7, 8], False)

In [3]:
num_words = len([comb for comb in product(['A', 'C', 'G', 'T'], repeat= 6)])

### CNN

In [13]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(num_words, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding1)
conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv1)
dropout1 = Dropout(0.5)(conv2)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)
flatten1 = Flatten()(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(flatten1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3334)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 3334, 32)          131072    
                                                                 
 conv1d_2 (Conv1D)           (None, 3332, 128)         12416     
                                                                 
 conv1d_3 (Conv1D)           (None, 3330, 128)         49280     
                                                                 
 dropout_2 (Dropout)         (None, 3330, 128)         0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1665, 128)        0         
 1D)                                                             
                                                           

In [14]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 96s 1s/step - loss: 1.8775 - Precision: 0.6586 - val_loss: 2.2077 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 93s 1s/step - loss: 1.1005 - Precision: 0.8003 - val_loss: 2.5978 - val_Precision: 0.3015 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 91s 1s/step - loss: 0.8725 - Precision: 0.8422 - val_loss: 1.9737 - val_Precision: 0.6481 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 93s 1s/step - loss: 0.7437 - Precision: 0.8895 - val_loss: 2.6162 - val_Precision: 0.5758 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - ETA: 0s - loss: 0.6683 - Precision: 0.9038
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 100s 2s/step - loss: 0.6683 - Precision: 0.9038 - val_loss: 2.6370 - val_Precision: 0.6620 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - 110s 2s

In [15]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 5s 237ms/step


,precision,recall,f1-score,support
snoRNA,0.392857,0.314286,0.349206,35.00000
rRNA,0.928571,0.942029,0.935252,138.00000
snRNA,0.714286,0.227273,0.344828,22.00000
mRNA,0.932642,0.967742,0.949868,186.00000
pre_miRNA,0.540984,0.660000,0.594595,50.00000
miRNA,0.456140,0.500000,0.477064,52.00000
tRNA,0.917526,1.000000,0.956989,89.00000
tmRNA,0.983051,0.828571,0.899225,70.00000
accuracy,0.828660,0.828660,0.828660,0.82866
macro avg,0.733257,0.679988,0.688378,642.00000


### CNN + BiLSTM

In [19]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(num_words, 32, input_length=max_len)(input_layer)
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding1)
dropout1 = Dropout(0.5)(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(dropout1)

lstm1 = Bidirectional(LSTM(128))(max_pool1)

dense1 = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))(lstm1)
dropout2 = Dropout(0.5)(dense1)
output_layer = Dense(8, activation='softmax')(dropout2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3334)]            0         
                                                                 
 embedding_3 (Embedding)     (None, 3334, 32)          131072    
                                                                 
 conv1d_5 (Conv1D)           (None, 3332, 128)         12416     
                                                                 
 dropout_6 (Dropout)         (None, 3332, 128)         0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1666, 128)        0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              263168    
 nal)                                                      

In [20]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit(train_data.seqs, train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 387s 6s/step - loss: 1.8407 - Precision: 0.9172 - val_loss: 3.3788 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 375s 6s/step - loss: 1.4214 - Precision: 0.8660 - val_loss: 3.5372 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 374s 6s/step - loss: 0.9649 - Precision: 0.8818 - val_loss: 2.5925 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - 364s 6s/step - loss: 0.6864 - Precision: 0.9003 - val_loss: 2.8219 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 346s 5s/step - loss: 0.4972 - Precision: 0.9024 - val_loss: 2.2147 - val_Precision: 0.0000e+00 - lr: 0.0010
Epoch 6/10
66/66 [==============================] - 340s 5s/step - loss: 0.3785 - Precision: 0.8915 - val_loss: 2.3799 - val_Precision: 0.2609 - lr: 0.0010
Epoch 7/10
66/66 [==========================

In [21]:
# Model prediction
model_pred = model.predict(test_data.seqs)
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 26s 1s/step


,precision,recall,f1-score,support
snoRNA,0.361111,0.371429,0.366197,35.000000
rRNA,0.909091,0.869565,0.888889,138.000000
snRNA,0.109589,0.363636,0.168421,22.000000
mRNA,0.830601,0.817204,0.823848,186.000000
pre_miRNA,0.681818,0.600000,0.638298,50.000000
miRNA,0.636364,0.403846,0.494118,52.000000
tRNA,0.976471,0.932584,0.954023,89.000000
tmRNA,0.785714,0.628571,0.698413,70.000000
accuracy,0.733645,0.733645,0.733645,0.733645
macro avg,0.661345,0.623355,0.629026,642.000000


### CNN + Features

In [26]:
input_layer = Input(shape=(max_len,))

embedding1 = Embedding(num_words, 32, input_length=max_len)(input_layer)
dropout1 = Dropout(0.2)(embedding1)
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu')(dropout1)
conv2 = Conv1D(filters=128, kernel_size=3, activation='relu')(conv1)
max_pool1 = MaxPooling1D(pool_size=2)(conv2)
dropout2 = Dropout(0.2)(max_pool1)
cnn_out = Flatten()(dropout2)

# feature extraction input

feat_extraction_input = Input(shape=(426,))
feat_extraction_out = Flatten()(feat_extraction_input)

concat = Concatenate()([cnn_out, feat_extraction_out])

dense1 = Dense(128, activation='relu')(concat)
dropout3 = Dropout(0.2)(dense1)
output_layer = Dense(8, activation='softmax')(dropout3)

model = Model(inputs=[input_layer, feat_extraction_input], outputs=output_layer)

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics= [tf.keras.metrics.Precision(name="Precision")])

model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 3334)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, 3334, 32)     131072      ['input_17[0][0]']               
                                                                                                  
 dropout_21 (Dropout)           (None, 3334, 32)     0           ['embedding_8[0][0]']            
                                                                                                  
 conv1d_16 (Conv1D)             (None, 3332, 128)    12416       ['dropout_21[0][0]']             
                                                                                            

In [27]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
]

model.fit([train_data.seqs, train_data.features], train_data.labels, batch_size=32, epochs=10, validation_split=0.1, callbacks=callbacks)

Epoch 1/10
66/66 [==============================] - 77s 1s/step - loss: 0.8487 - Precision: 0.8471 - val_loss: 1.0698 - val_Precision: 0.7535 - lr: 0.0010
Epoch 2/10
66/66 [==============================] - 68s 1s/step - loss: 0.4297 - Precision: 0.8938 - val_loss: 0.6657 - val_Precision: 0.9137 - lr: 0.0010
Epoch 3/10
66/66 [==============================] - 69s 1s/step - loss: 0.3022 - Precision: 0.9143 - val_loss: 0.7356 - val_Precision: 0.8164 - lr: 0.0010
Epoch 4/10
66/66 [==============================] - ETA: 0s - loss: 0.1908 - Precision: 0.9457
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
66/66 [==============================] - 70s 1s/step - loss: 0.1908 - Precision: 0.9457 - val_loss: 1.6883 - val_Precision: 0.6209 - lr: 0.0010
Epoch 5/10
66/66 [==============================] - 70s 1s/step - loss: 0.1016 - Precision: 0.9773 - val_loss: 1.2246 - val_Precision: 0.7534 - lr: 2.0000e-04
Epoch 6/10
66/66 [==============================] - ETA: 0s 

In [28]:
# Model prediction
model_pred = model.predict([test_data.seqs, test_data.features])
y_pred = np.argmax(model_pred, axis=1)
y_true = np.argmax(test_data.labels, axis=1)

# Classification report
report = classification_report(y_true, y_pred, target_names=test_data.names, output_dict=True)
pd.DataFrame(report).T

21/21 [==============================] - 4s 206ms/step


,precision,recall,f1-score,support
snoRNA,0.657895,0.714286,0.684932,35.000000
rRNA,0.910959,0.963768,0.936620,138.000000
snRNA,0.812500,0.590909,0.684211,22.000000
mRNA,0.936842,0.956989,0.946809,186.000000
pre_miRNA,0.673077,0.700000,0.686275,50.000000
miRNA,0.591837,0.557692,0.574257,52.000000
tRNA,0.967033,0.988764,0.977778,89.000000
tmRNA,1.000000,0.857143,0.923077,70.000000
accuracy,0.873832,0.873832,0.873832,0.873832
macro avg,0.818768,0.791194,0.801745,642.000000
